# Testing Optuna

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 08/07/2025   | Martin | Create  | Notebook created to test optuna for hyperparameter search | 

# Content

* [Vanilla Optuna](#header-1)

# Vanilla Optuna

In [1]:
%load_ext watermark

In [9]:
import optuna
import lightgbm as lgb
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

In [5]:
# Load data
f = pl.read_csv("data/pca_120.csv")
f.head()

pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,pca_24,pca_25,pca_26,pca_27,pca_28,pca_29,pca_30,pca_31,pca_32,pca_33,pca_34,pca_35,pca_36,pca_37,…,pca_86,pca_87,pca_88,pca_89,pca_90,pca_91,pca_92,pca_93,pca_94,pca_95,pca_96,pca_97,pca_98,pca_99,pca_100,pca_101,pca_102,pca_103,pca_104,pca_105,pca_106,pca_107,pca_108,pca_109,pca_110,pca_111,pca_112,pca_113,pca_114,pca_115,pca_116,pca_117,pca_118,pca_119,pca_120,timestamp,label
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
-8.704309,5.45448,0.089441,-7.54084,4.768329,-5.736881,-0.231744,-1.788746,2.788266,-1.190636,3.148074,3.06826,6.837452,0.155804,-0.885875,-2.395842,3.13128,-4.266488,0.943026,0.60447,0.816785,-0.961067,-3.641479,-0.078829,-4.117839,4.6193,0.883802,-0.229885,0.600305,3.098696,-2.115549,1.951743,-0.103804,1.010359,1.52759,3.009872,0.625632,…,-0.83295,0.411902,1.958133,0.40939,0.075597,-1.114706,-0.102757,-0.103655,-0.173762,-0.167699,0.303377,-1.386903,0.9592,0.333935,-0.322841,0.100387,-0.00093,0.360481,-0.643133,-0.73806,0.887927,0.507611,-1.714703,-0.139313,-0.649024,-1.111647,-0.281171,0.62976,-1.064831,0.435178,0.168905,-0.654007,0.287436,-0.9085,-0.483989,"""2023-03-01T00:00:00.000000000""",0.562539
-7.994975,5.527279,-0.093258,-4.78567,4.097361,-6.208308,0.004089,-0.355855,3.798541,0.215877,2.648574,3.351467,6.158131,0.254169,-3.347221,-2.473193,3.618805,-3.161974,3.224999,2.054594,-2.296999,0.672556,-1.778871,-1.52334,1.743752,4.333566,-0.902184,0.029043,0.229918,2.181485,0.296175,1.67046,-0.227987,1.403915,2.228776,2.469899,-0.592644,…,-1.408637,0.337922,0.350379,-0.137153,0.000447,-1.546396,0.222861,0.54616,-0.286338,0.611064,-0.42617,-0.987761,1.431453,0.403918,-0.030256,0.113834,0.394106,-0.945749,-0.211975,-0.393696,1.696762,-0.206892,-2.071071,-0.696854,-0.789737,-0.818696,-0.342221,0.566906,-0.331511,0.761192,0.021027,-0.536371,-0.425608,0.056954,-0.552911,"""2023-03-01T00:01:00.000000000""",0.533686
-8.26279,5.052642,-0.215007,-4.022985,3.920973,-5.534213,-0.192796,1.0202,3.404864,1.268972,2.083842,3.41862,5.318265,-0.448951,-3.372344,-2.696721,3.405499,-1.909767,3.035536,2.726676,-2.895715,0.675317,-2.672765,-1.462927,4.68873,3.224458,-1.63386,-0.420518,0.737464,1.673416,1.288468,1.626087,-0.205033,1.21317,2.613173,1.760396,-0.513233,…,-1.435126,-0.602025,0.566827,0.197806,0.759271,-1.606877,-0.002562,-0.168767,-0.177104,-0.062822,0.693314,-1.088589,1.134069,-0.634355,0.603804,1.470887,1.181964,1.050156,0.296225,-0.416962,1.836099,-0.317207,-1.092388,-1.08504,-0.835654,-0.032517,-0.704754,-0.151475,0.02618,0.294466,-0.014631,-0.29039,-0.119407,-0.631851,-0.651055,"""2023-03-01T00:02:00.000000000""",0.546505
-6.583707,7.511057,-1.011146,-5.043764,4.93578,-5.846506,0.945893,-1.987165,5.037395,-0.61052,3.062758,3.185566,6.651355,0.209246,-4.044023,-2.209972,2.891386,-3.855405,3.003505,0.468633,-1.329728,0.101761,-1.862309,-2.455378,2.926416,4.623982,-2.465403,-0.558694,1.26826,1.792994,1.559252,2.209627,-0.713931,1.75982,1.519249,3.20797,-0.787381,…,-0.60509,-0.308904,0.684425,-0.057665,0.137066,-1.708174,-0.534534,0.067984,-0.612431,0.29722,0.05843,-1.00365,1.514464,-0.185545,0.080562,1.241688,0.870678,-0.031129,0.022311,-0.28331,1.933081,0.522886,-0.72863,-0.696465,-0.941094,-0.560078,-0.153947,1.394604,-0.566123,1.238746,0.415802,-0.680394,-0.322317,-0.24378,-0.658033,"""2023-03-01T00:03:00.000000000""",0.357703
-6.496762,7.126301,-0.582182,-5.279062,4.686467,-6.359052,0.802733,-0.713604,4.994975,0.220712,3.183836,2.947917,5.852149,-0.284797,-3.676434,-2.022073,2.813884,-2.542805,3.164651,2.014087,-1.716045,0.661284,-1.617872,-1.461303,2.399019,3.573622,-2.939042,-0.593127,1.061588,1.461763,1.08

In [14]:
# Process data
df = f.drop('timestamp')
y = df['label'].to_numpy()
X = df.drop('label').to_numpy()

In [24]:
# Define Optuna hyperparameter search
def objective(trial):
  train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2)
  dtrain = lgb.Dataset(train_X, label=train_y)

  params = {
    "objective": "regression",
    "metric": trial.suggest_categorical("metric", ['rmse', 'l1', 'l2']),
    "verbosity": -1,
    "boosting_type": "gbdt",
    "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
    "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
    "num_leaves": trial.suggest_int("num_leaves", 2, 256),
    "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
    "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
    "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
    "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
  }

  gbm = lgb.train(params, dtrain)
  preds = gbm.predict(valid_X)
  correlation = pearsonr(preds, valid_y).statistic
  return correlation

In [25]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-07-08 21:54:48,884] A new study created in memory with name: no-name-8fb33470-b4b8-4e95-847b-39412106b769
[I 2025-07-08 21:54:55,463] Trial 0 finished with value: 0.7078976207768308 and parameters: {'metric': 'rmse', 'lambda_l1': 0.0015228340929400036, 'lambda_l2': 5.6259170692037115, 'num_leaves': 136, 'feature_fraction': 0.9672049075822554, 'bagging_fraction': 0.6481347419311873, 'bagging_freq': 3, 'min_child_samples': 34}. Best is trial 0 with value: 0.7078976207768308.
[I 2025-07-08 21:55:02,252] Trial 1 finished with value: 0.7579786635086418 and parameters: {'metric': 'l1', 'lambda_l1': 0.17406950446260772, 'lambda_l2': 0.0042725753222355905, 'num_leaves': 224, 'feature_fraction': 0.6349210448507527, 'bagging_fraction': 0.5473132297241596, 'bagging_freq': 4, 'min_child_samples': 29}. Best is trial 1 with value: 0.7579786635086418.
[I 2025-07-08 21:55:08,278] Trial 2 finished with value: 0.7607867087440019 and parameters: {'metric': 'l1', 'lambda_l1': 0.0013125804905226085

Number of finished trials: 10
Best trial:
  Value: 0.7677109933499279
  Params: 
    metric: l1
    lambda_l1: 2.4807017776968476
    lambda_l2: 0.25835486219950426
    num_leaves: 254
    feature_fraction: 0.632086482233723
    bagging_fraction: 0.6011428545820632
    bagging_freq: 3
    min_child_samples: 88


AttributeError: 'Study' object has no attribute 'best_score'

In [2]:
%watermark

Last updated: 2025-07-08T21:35:04.500769+08:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 9.4.0

Compiler    : Clang 13.0.0 (clang-1300.0.29.30)
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit

